In [ ]:
!pip install beautifulsoup4
!pip install streamlit
!pip install jq
!pip install -U transformers
!pip install -q bitsandbytes torch accelerate
!pip install langchain faiss-cpu sentence-transformers langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [118]:
import requests
import warnings
warnings.simplefilter('ignore')
import re
from datetime import date
from bs4 import BeautifulSoup
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM , AutoTokenizer , BitsAndBytesConfig , pipeline ,VitsModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser , JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain_community.llms import HuggingFaceEndpoint
from huggingface_hub import login
from langchain_community.document_loaders import JSONLoader
import torch
from getpass import getpass
import os
import json

In [ ]:
login()

In [ ]:
home_page_url = "https://ticka.world/shop/"
sub_pages_url="https://ticka.world/shop/page/"
home_page = requests.get(home_page_url)
sub_page = requests.get(home_page_url)
soup = BeautifulSoup(home_page.content, "html.parser")

In [ ]:
results = soup.find("div",class_="products layout-grid grid-columns-4 grid-columns--tablet3 grid-columns--mobile2")

In [ ]:
def preprocessProductName(productName):
  productName= productName.lower().replace('–','').replace('  ',' ').split(' ')
  sep='-'
  res=sep.join(productName)
  return res

In [ ]:
def removeWhiteSpaces(ch):
    return " ".join(ch.split())
def getProductsInfo(productName):
  productUrl=f'https://ticka.world/product/{productName}'
  page=requests.get(productUrl)
  if page.status_code !=200 :
    return "",""
  scoup=BeautifulSoup(page.content,"html.parser")
  infoDiv=scoup.find("ul",class_="list-unstyled")
  details=infoDiv.find_all("span",class_="details")
  rate=infoDiv.find("li",class_="clearfix")
  prefix=["Store Name : ","vendor Name : ","Adresse : "]
  for i in range(len(details)) :
    details[i]=prefix[i]+removeWhiteSpaces(details[i].text)
  rate=removeWhiteSpaces(rate.text)
  return details ,rate

In [ ]:
def fetchItems(results,products):
  job_elements = results.find_all("div",class_="product-info")
  for i  in tqdm(range(len(job_elements))) :
    title_element = job_elements[i].find("h2", class_="woocommerce-loop-product__title")
    price=job_elements[i].find("span",class_="woocommerce-Price-amount amount")
    price=re.sub('[^0-9.]','',str(price))+" TND"
    title_element=preprocessProductName(title_element.text)
    info,rate=getProductsInfo(title_element)
    products.append({"product":title_element,"price":price,"info":info,"rate":rate})
  return products

In [ ]:
def getItems(products):
    for num_page in range(1,16):
        sub_page = requests.get(sub_pages_url+str(num_page))
        scoup = BeautifulSoup(sub_page.content, "html.parser")
        results = scoup.find("div",class_="products layout-grid grid-columns-4 grid-columns--tablet3 grid-columns--mobile2")
        products=fetchItems(results,products)
    return products

In [ ]:
products=getItems([])

100%|██████████| 14/14 [01:03<00:00,  4.53s/it]


In [ ]:
products[0]

{'product': 'hoodie-you-make-my-earth-quake',
 'price': '73.00. TND',
 'info': ['Store Name : Tinbri Tounsi',
  'vendor Name : Tinbri Tounsi',
  'Adresse : Tunisia SousahRiadhSousse4023'],
 'rate': 'No ratings found yet!'}

In [ ]:
def getTickaInformations(tickaInformation):
  landingPageUrl="https://ticka.world/"
  page=requests.get(landingPageUrl)
  scoup = BeautifulSoup(page.content,"html.parser")
  questions=scoup.find_all("a",class_="haru-accordion-title")
  result=scoup.find_all("div",class_="haru-tab-content haru-clearfix")
  activeItem=scoup.find("div",class_="haru-tab-content haru-clearfix active")
  full_text="Question : "+questions[0].text+"\nReponse : "+activeItem.find("span").text
  tickaInformation.append({"content":full_text})
  for i in tqdm(range(len(result))):
    full_text="Question : "+questions[i+1].text+"\nReponse : "+result[i].find("span").text
    tickaInformation.append({"content":full_text})
  return tickaInformation

In [ ]:
tickaInformation=getTickaInformations([])

100%|██████████| 11/11 [00:00<00:00, 7347.88it/s]


In [ ]:
products.append({"Informations":tickaInformation})
prod={"storeInfo":products}

In [ ]:
data=json.dumps(prod)

In [ ]:
with open("data.json","w") as file :
    file.write(data)

In [28]:
def tokenize(model_repo):
    tokenizer=AutoTokenizer.from_pretrained(model_repo)
    # tokenizer.pad_token = tokenizer.eos_token
    # tokenizer.padding_side = "right"
    return tokenizer

def uploadModel(tokenizer,model_repo):
  # compute_dtype = getattr(torch, "float16")
  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_compute_dtype=compute_dtype,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_use_double_quant=False,
    )
  model=AutoModelForCausalLM.from_pretrained(model_repo,
                             quantization_config=bnb_config)
  llm=pipeline(
    model=model,
    task="text-generation",
    tokenizer=tokenizer,
    temperature=0.6,
    return_full_text=True,
    do_sample=True,
    max_new_tokens=1000,
    )
  mistral = HuggingFacePipeline(pipeline=llm)
  return mistral

In [ ]:
def split_text(json_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=250, chunk_overlap=30
    )
    doc=text_splitter.split_documents(json_data)
    return doc
def retiverFaiss(json_data):
    doc=split_text(json_data)
    db=FAISS.from_documents(
        doc,
        HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    )
    retiver=db.as_retriever()
    return retiver

In [61]:
def getPrompt(template):
    prompt=PromptTemplate(
        input_variables=["question"],
        template=template,
    )
    return prompt

In [128]:
def getCompletion(prompt,retiver,model,question):
    parser = StrOutputParser()
    rag_chain = ({
        "data":retiver,
        "question":RunnablePassthrough()
    }|prompt
    |model
    |parser
  )
    result=rag_chain.invoke(question)
    return result


In [120]:
def loadJsonData():
    loader = JSONLoader(
        file_path='/content/data.json',
        jq_schema='.storeInfo[]',
        text_content=False
    )
    return loader.load()

In [ ]:
jsonData=loadJsonData()

In [ ]:
retiver=retiverFaiss(jsonData)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Mistral

In [ ]:
model_repo ="mistralai/Mistral-7B-Instruct-v0.2"
tokenizer=tokenize(model_repo)
model=uploadModel(tokenizer,model_repo)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
template="""Instruction : [INST]You are a helpful AI system, which answers the user's question for an e-commerce startup based on the relevant data provided to you.
Instructions :
Try to formulate your Responses.
Answer the user's question without providing any Explanation.
If you don't know the answer, just say I don't know.
Contextual Comprehension: Unravel the meaning and tone.

Here is the data you need :
{data}
User Question :
{question}
[/INST]
"""
prompt=getPrompt(template)

In [ ]:
results=getCompletion(prompt,retiver,model,question="How can i contact ticka ? ")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(results)

You can reach Ticka's customer support team through their website or by emailing customer-service@ticka.com.


# Gemma

In [45]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] =getpass()

··········


In [121]:
model_repo= "google/gemma-7b-it"
model=HuggingFaceEndpoint(
    repo_id=model_repo , max_length=1024, temperature=0.8
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [193]:
template="""system \nYou are a helpful AI system, which answers the user's question for an e-commerce startup based on the relevant data provided to you.
Instructions :
Answer the user's question without providing any Explanation.
Contextual Comprehension: Unravel the meaning and tone.

Here is the data you need :
{data}

user quesion :
{question}

Answer :
"""

In [194]:
prompt=getPrompt(template)
results=getCompletion(prompt,retiver,model,question="what is ticka ?")

In [195]:
print(results)

Ticka is an online platform that connects users with local artists, allowing them to create and purchase customized products with unique designs.
